# SETUP

## Import Libraries

In [77]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Read MongoDB

In [78]:
client = MongoClient('mongodb://192.168.68.117:27017')
db = client["test"]

## Initialize Output DataFrame

In [79]:
output_df = pd.DataFrame()

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [80]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [ ]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [82]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [83]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

### Complete Range of All Matches

In [84]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [85]:
output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

output_df["Auto Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Auto Speaker Notes Total Avg"] = sum([output_df.at[team, "Auto Speaker Notes Near Avg"], output_df.at[team, "Auto Speaker Notes Mid Avg"], output_df.at[team, "Auto Speaker Notes Far Avg"]])

In [86]:

grouped = matchapps_df.groupby("metadata.robotTeam")
output_df["Auto Speaker Notes Near Max"] = grouped["autoNotes.near"].max()
output_df["Auto Speaker Notes Mid Max"] = grouped["autoNotes.mid"].max()
output_df["Auto Speaker Notes Far Max"] = grouped["autoNotes.far"].max()
output_df["Auto Speaker Notes Total Max"] = (matchapps_df[["autoNotes.far", "autoNotes.mid", "autoNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max
metadata.robotTeam,,,,,,,,
4,0.714286,0.000000,0.000000,0.714286,2,0,0,2
498,0.250000,0.125000,0.000000,0.375,1,1,0,1
696,0.428571,0.428571,0.000000,0.857143,1,1,0,2
973,0.000000,0.000000,0.000000,0.0,0,0,0,0
980,0.555556,0.000000,0.000000,0.555556,1,0,0,1
1148,0.000000,0.000000,0.000000,0.0,0,0,0,0
1159,0.000000,0.000000,0.000000,0.0,0,0,0,0
1165,0.000000,0.000000,0.000000,0.0,0,0,0,0
1197,0.142857,0.000000,0.000000,0.142857,1,0,0,1


#### Auto Amp Notes Average

In [87]:
output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [88]:
output_df["Auto Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Auto Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Auto Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].max()

/tmp/ipykernel_113491/457572201.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
/tmp/ipykernel_113491/457572201.py:12: RuntimeWarning: invalid value encountered in scalar divide
  output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes


#### Auto All Notes Average

In [89]:
output_df["Auto Notes Avg"] = (output_df["Auto Notes Amp Avg"] +
                               output_df["Auto Speaker Notes Near Avg"] +
                               output_df["Auto Speaker Notes Mid Avg"] +
                               output_df["Auto Speaker Notes Far Avg"]) / 4

#### Average Auto points

In [90]:
output_df['Auto Points Avg'] = (5 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 2 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [91]:
output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

output_df["Tele Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Tele Speaker Notes Total Avg"] = sum([output_df.at[team, "Tele Speaker Notes Near Avg"], output_df.at[team, "Tele Speaker Notes Mid Avg"], output_df.at[team, "Tele Speaker Notes Far Avg"]])

In [92]:
output_df["Tele Speaker Notes Near Max"] = grouped["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = grouped["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = grouped["teleNotes.far"].max()
output_df["Tele Speaker Notes Total Max"] = (matchapps_df[["teleNotes.far", "teleNotes.mid", "teleNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Auto Notes Avg,Auto Points Avg,Tele Speaker Notes Near Avg,Tele Speaker Notes Mid Avg,Tele Speaker Notes Far Avg,Tele Speaker Notes Total Avg,Tele Speaker Notes Near Max,Tele Speaker Notes Mid Max,Tele Speaker Notes Far Max,Tele Speaker Notes Total Max
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
4,0.714286,0.000000,0.000000,0.714286,2,0,0,2,0.000,0.2,...,0.178571,17.142857,3.285714,0.142857,0.000000,3.428571,9,1,0,9
498,0.250000,0.125000,0.000000,0.375,1,1,0,1,0.000,2.5,...,0.093750,14.875000,1.125000,1.500000,0.000000,2.625,4,6,0,8
696,0.428571,0.428571,0.000000,0.857143,1,1,0,2,0.000,0.666667,...,0.214286,19.857143,0.714286,3.142857,0.000000,3.857143,3,11,0,14
973,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,43.375000,2.625000,5.750000,0.250000,8.625,9,12,2,13
980,0.555556,0.000000,0.000000,0.555556,1,0,0,1,0.000,0.6,...,0.138889,3.888889,0.777778,0.000000,0.000000,0.777778,4,0,0,4
1148,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,13.777778,0.333333,0.777778,0.000000,1.111111,2,3,0,3
1159,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,NaN,...,0.000000,2.142857,0.285714,0.142857,0.000000,0.428571,1,1,0,2
1165,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,2.777778,0.555556,0.000000,0.000000,0.555556,2,0,0,2
1197,0.142857,0.000000,0.000000,0.142857,1,0,0,1,0.000,2.0,...,0.035714,10.285714,1.714286,0.000000,0.000000,1.714286,7,0,0,7


#### Teleop Amp Notes Average

In [93]:
output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

 # CHANGE FORMAT LATER
output_df["Tele Max Amp Notes"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].max()

#### Teleop All Notes Average

In [94]:
output_df["Tele Notes Avg"] = (output_df["Tele Notes Amp Avg"] +
                               output_df["Tele Speaker Notes Near Avg"] +
                               output_df["Tele Speaker Notes Mid Avg"] +
                               output_df["Tele Speaker Notes Far Avg"]) / 4

 # CHANGE FORMAT LATER
output_df["Tele Max Notes"] = None

#### Teleop Percent of Missed Notes

In [95]:
output_df["Tele Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Tele Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Tele Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].max()

/tmp/ipykernel_113491/471659397.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
/tmp/ipykernel_113491/471659397.py:12: RuntimeWarning: invalid value encountered in scalar divide
  output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes


In [96]:
output_df["Tele Speaker Notes Near Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].max()

#### Average Teleop Points

In [97]:
output_df['Tele Points Avg'] = (2 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 1 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

In [98]:
# output_df['Endgame Points Avg'] =
#(np.where(matchapps_df['trapNotes']), 0, 1)
np.where(matchapps_df['climb'].isin(['source', 'center', 'amp']), 0, 3)


array([3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 3, 0, 3, 0, 0, 0, 3, 3, 0, 0, 3, 0, 3, 3, 3, 0, 3, 3, 3,
       3, 3, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3,
       3, 0, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 0, 0, 3, 0, 3, 3,
       3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3, 3,
       3, 0, 3, 3, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0,
       3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 0, 3,
       3, 0, 0, 3, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3,

### Other Data Analysis Calculations

#### Trap Notes

In [99]:
output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [100]:
output_df["Percent of Successful Climbs"] = None
output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

#### Percent of Harmonized Climbs out of Successful Climbs

In [101]:
# output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

# output_df.index

# for team in team_list:
#     output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = output_df.at[team, "Harmonized Climbs Total"] / output_df.at[team, "Successful Climbs Total"]

#### Percent of Times Robot Left Starting Zone During Autos

In [102]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [103]:
# output_df["Endgame Custom Metric"] = ((output_df["Percent of Successful Climbs"] * 5) ** 2 + (output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (output_df["Trap Notes Avg"] * 5) ** 2) / 3
# output_df["Endgame Custom Metric"] = output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [104]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [ ]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

## Superapps Data Analysis

### Average Fouls per Match for Different Foul Types

In [106]:
output_df["Average In Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.inBot"].mean()
output_df["Average Damage Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.damageBot"].mean()
output_df["Average Tip Entangle Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.tipEntangBot"].mean()
output_df["Average Pin Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinBot"].mean()
output_df["Average Podium Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.podiumFoul"].mean()
output_df["Average Zone Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.zoneFoul"].mean()
output_df["Average Stage Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.stageFoul"].mean()
output_df["Average Over Exc Chute Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.overExtChute"].mean()

### Average Total Fouls per Match

In [108]:
output_df["Average Fouls per Match"] = None

for team in team_list:
    output_df.at[team, "Average Fouls per Match"] = sum([
        output_df.at[team, "Average In Bot Fouls per Match"]
        + output_df.at[team, "Average Damage Bot Fouls per Match"]
        + output_df.at[team, "Average Tip Entangle Bot Fouls per Match"]
        + output_df.at[team, "Average Pin Bot Fouls per Match"]
        + output_df.at[team, "Average Podium Fouls per Match"]
        + output_df.at[team, "Average Zone Fouls per Match"]
        + output_df.at[team, "Average Stage Fouls per Match"]
        + output_df.at[team, "Average Over Exc Chute Fouls per Match"]])

### Percent of Different Defense Types in Matches AND Main Defense Type

In [109]:
output_df["Percent of Matches with No Defense"] = None
output_df["Percent of Matches with Some Defense"] = None
output_df["Percent of Matches with Full Defense"] = None
output_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = superapps_df.loc[superapps_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    output_df.at[team, "Percent of Matches with No Defense"] = no_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Some Defense"] = some_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Full Defense"] = full_defense_count / defense_total_count
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    output_df.at[team, "Main Defense Type"] = main_defense_type

## Additionals

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Average Podium Fouls per Match,Average Zone Fouls per Match,Average Stage Fouls per Match,Average Over Exc Chute Fouls per Match,Average Fouls per Match,Percent of Matches with No Defense,Percent of Matches with Some Defense,Percent of Matches with Full Defense,Main Defense Type,teamNumber
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
4,0.714286,0.000000,0.000000,0.714286,2,0,0,2,0.000,0.2,...,0.000000,0.142857,0.000000,0.000000,0.142857,0.714286,0.285714,0.0,No Defense,4
498,0.250000,0.125000,0.000000,0.375,1,1,0,1,0.000,2.5,...,0.000000,0.285714,0.000000,0.000000,0.571429,0.714286,0.142857,0.142857,No Defense,498
696,0.428571,0.428571,0.000000,0.857143,1,1,0,2,0.000,0.666667,...,0.000000,0.000000,0.000000,0.000000,0.428571,0.857143,0.0,0.142857,No Defense,696
973,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,0.000000,0.000000,0.000000,0.111111,0.666667,0.333333,0.0,No Defense,973
980,0.555556,0.000000,0.000000,0.555556,1,0,0,1,0.000,0.6,...,0.000000,0.000000,0.000000,0.000000,0.0,0.888889,0.111111,0.0,No Defense,980
1148,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,0.000000,0.125000,0.000000,0.125,0.875,0.125,0.0,No Defense,1148
1159,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,NaN,...,0.000000,0.000000,0.000000,0.250000,0.25,0.75,0.125,0.125,No Defense,1159
1165,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.200000,0.200000,0.000000,0.000000,0.6,0.8,0.2,0.0,No Defense,1165
1197,0.142857,0.000000,0.000000,0.142857,1,0,0,1,0.000,2.0,...,0.000000,0.000000,0.000000,0.166667,0.166667,1.0,0.0,0.0,No Defense,1197


# FINALIZE AND SEND DATA

## Send Data

In [110]:
output_df['teamNumber'] = output_df.index

In [111]:
output_df.to_json("../server/static/output_analysis.json", orient='records', default_handler=str)
output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Average Podium Fouls per Match,Average Zone Fouls per Match,Average Stage Fouls per Match,Average Over Exc Chute Fouls per Match,Average Fouls per Match,Percent of Matches with No Defense,Percent of Matches with Some Defense,Percent of Matches with Full Defense,Main Defense Type,teamNumber
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
4,0.714286,0.000000,0.000000,0.714286,2,0,0,2,0.000,0.2,...,0.000000,0.142857,0.000000,0.000000,0.142857,0.714286,0.285714,0.0,No Defense,4
498,0.250000,0.125000,0.000000,0.375,1,1,0,1,0.000,2.5,...,0.000000,0.285714,0.000000,0.000000,0.571429,0.714286,0.142857,0.142857,No Defense,498
696,0.428571,0.428571,0.000000,0.857143,1,1,0,2,0.000,0.666667,...,0.000000,0.000000,0.000000,0.000000,0.428571,0.857143,0.0,0.142857,No Defense,696
973,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,0.000000,0.000000,0.000000,0.111111,0.666667,0.333333,0.0,No Defense,973
980,0.555556,0.000000,0.000000,0.555556,1,0,0,1,0.000,0.6,...,0.000000,0.000000,0.000000,0.000000,0.0,0.888889,0.111111,0.0,No Defense,980
1148,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.000000,0.000000,0.125000,0.000000,0.125,0.875,0.125,0.0,No Defense,1148
1159,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,NaN,...,0.000000,0.000000,0.000000,0.250000,0.25,0.75,0.125,0.125,No Defense,1159
1165,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000,inf,...,0.200000,0.200000,0.000000,0.000000,0.6,0.8,0.2,0.0,No Defense,1165
1197,0.142857,0.000000,0.000000,0.142857,1,0,0,1,0.000,2.0,...,0.000000,0.000000,0.000000,0.166667,0.166667,1.0,0.0,0.0,No Defense,1197


In [112]:
client.close()